# Clean data in steps
* The original data "Tianlin_GFR_CKD_EPI_31082020.xls" resides in `project_calc/data/original`.
<br>It contains 14345 rows and 85 columns.
* The disease group "disease_group_SamHobson_PS_SH.xlsx" resides in `project_calc/data/original`.
* The NTCVD info from GB "Tianlin_ntcvd_all_checkbyGB.xlsx" and "checkedByGB_Tianlin_ntcvd.xlsx" reside in `project_calc/data/original`.

## python-class CLEAN1
1. `x1=CLEAN1(f1+'Tianlin_GFR_CKD_EPI_31082020.xls')` load the data 
    * $\Rightarrow$ Running `x1=CLEAN1(f1+'Tianlin_GFR_CKD_EPI_31082020.xls')` displays 'It contains 14345 rows and 85 columns'
    * uncomment to see column names
    * uncomment to see study names
    
1. `remove_nan` to remove one patient with nan value in eGFR
    * $\Rightarrow$ Running `x1.remove_nan()` displays: 'It contains 14344 rows and 85 columns'</tab>    
   
2. `remove_study` to remove studies with 'fidStudie' in `['FROG_ICU','CIHF','PCHF','MTWA','Haubitz_geblindet_II','Stroke']`
    * $\Rightarrow$ Running `x1.remove_study()` displays 'It contains 10650 rows and 85 columns'</tab>
    * 'FROG_ICT' consists of ICU patients which have atypical peptidomics
    * 'CIHF', 'PCHF','MTWA': Harald suggested me removing them
    * 'Haubitz_geblindet_II': it is a blinded study with unknown patients' data
    * 'Stroke': with incredible eGFR
3. `remove_s_creatinine_bySam` to remove 4 patients
    * Sam pointed out they had rocket high serum creatinine values (email on 20200911)
4. `remove_dialysis` to remove patients underwent dialysis
    * input1=`'checkedByGB_Tianlin_ntcvd.xlsx'` provided by GB
    * those are **NTCVD** patients only
3. `remove_age_below18` to remove patients age below 18
    * because they have healthier peptidomics albeit low eGFR
4. `remove_egfr_above` to remove patients with eGFR above an arbitrary number n
    * $\Rightarrow$ Running `x1.remove_egfr_above(150)` displays 'It contains 8128 rows and 85 columns'
4. `gfr_predictionHBI` to correct eGFR values for patients in studie 'Predictions_HBI'.It is known that some patients bear the wrong serum creatinine unit. For patiens with extra smaller serum creatinine (<0.1), I:
    1. multiply serum creatinine with 88.42 to convert mmol/L to mg/dL
    2. apply the EPI formula to calculate eGFR for male and female (we assume all are non-black)
    3. compare the eGFR with the website [calculate-by-Qxmd](https://qxmd.com/calculate/calculator_251/egfr-using-ckd-epi)
    4. $\Rightarrow$ Running `x1.gfr_predictionHBI()` displays no output (because it does not remove/add rows, but you can uncomment the `print` line to see the corrected eGFR and serum creatinine
5. `merge_diseases` to merge subdiseases into bigger group
    * input= `f1+'disease_group_SamHobson_PS_SH.xlsx'
    * it first merge subgroups into a bigger groups
    * then it removes the subgroups that have been merged
    * $\Rightarrow$ Running `x1.merge_diseases()` displays 'It contains 8128 rows and 18 columns'
6. `exclude_diseases` to exclude patients with value "1.0" in the "Exclude" column
    * $\Rightarrow$ Running `x1.exclude_diseases()` displays 'It contains 7953 rows and 17 columns'
7. `update_Calcification` to update diseases according to data given by Sam
    * update CVD and diabetes
    * all patients from "Calcification" were assigned "1" for CKD
7. `insert_col_from_sql` insert columns from files queried based on my own sql script `project_calc/sql/calc_20200915.txt`
    ```
    #urinary creatinine
    SELECT    o.Probe_id, pa.Parameter, pa.Einheit, p.Datum, p.Wert
    FROM         dbo.tblP_Parameter p INNER JOIN
                          dbo.tblOriginalproben o ON p.fidPatient = o.fidPatient INNER JOIN
                          dbo.tblParameter pa ON p.fidParameter = pa.idParameter
    WHERE     (p.fidParameter IN (9,47,157))

    #ACR
    SELECT    o.Probe_id, pa.Parameter, pa.Einheit, p.Datum, p.Wert
    FROM         dbo.tblP_Parameter p INNER JOIN
                          dbo.tblOriginalproben o ON p.fidPatient = o.fidPatient INNER JOIN
                          dbo.tblParameter pa ON p.fidParameter = pa.idParameter
    WHERE     (p.fidParameter=56)
    ```
    * `x1.insert_col_from_sql(f1+'u_creatinine_unit_date.xls','U_Kreatinin')` inserts urinary creatinine and its units (notice that it carries two units, "mg/dL" and "mmol/l").
    * `x1.insert_col_from_sql(f1+'acr_unit_date.xls','acr')` inserts urinary albumin to creatinine ratio. It carries one unit, "mg/g"
    
7. `update_NTCVD` to update **NTCVD** patients according to data given by GB
    * input1=`'Tianlin_ntcvd_all_checkbyGB.xlsx'`
    * it updates columns "U_Albumin" and "acr"
    * it displays the number of non-nan rows of "U_Albumin" and "acr" before and after update
    * uncomment to print "U_Albumin" and "acr" of NTCVD patients
7. `replace_0_by_nan` to replace 0 by nan for any arbitrary column
    * we replace 0 in `U_Albumin`
8. `replace_0_by_min` to replace 0 by the smallest non-zero value in the same study
    * we replace 0 in `U_Albumin` by running `x1.replace_0_by_min('U_Albumin')`, you can theoretically do it with any column name
    * It is a group-wise operation: e.g., 
        * 0 in study='CVD' will be replaced by 0.5, which is the smallest record in 'CVD', 
        * while 0 in study='FSGF' will be replaced by 1.8, which is the smallest record in 'FSGF'
7. `assign_sex` to convert "gender" to boolean values
    * "männlich" to 1
    * "weblich" to 0
    * $\Rightarrow$ Running `x1.assign_sex()` displays no output 
8. `albuminuria_group` to assign normal/micro/macro albuminuria, cut at 30 and 300 mg/g
    * create three columns `['normalalbuminuria', 'microalbuminuria', 'macroalbuminuria']` with boolean values
9. `egfr_group` to assign eGFR group according to Sam, cut at arbitrary thresholds
    * create one columns `egfr_group` with group labels 
    * larger the eGFR, smaller the group label
6. `export_excel` to export the cleaned table to an .xlsx file.
    * $\Rightarrow$ Running `x1.export_excel(path/name)` displays the path and name of the cleaned file.

    
### Remarks: eGFR is given by the formula
$GFR = 141 * \min(Scr/κ,1)^{\alpha}* \max(Scr/κ, 1)^{-1.209} * 0.993^{Age} * 1.018 [if female] * 1.159 [if black]$
* Scr is serum creatinine (mg/dL), 
* κ is 0.7 for females and 0.9 for males, 
* α is -0.329 for females and -0.411 for males, 
* min indicates the minimum of Scr/κ or 1, and max indicates the maximum of Scr/κ or 1.

### To note: 
f1 and f2 are location+file name of original (input) and cleaned (output) data respectively. They were named according to my own folder setting. (I stored the original and cleaned file at two places so that I do not mix them up). 
<br>It is completely optional to use the same folder setting as mine, but in case you want, below is my setting:
* I created a main folder called "project_calc"
    * "project_calc" has two subfolders, called "python" and "data" respectively
    * "data" has two subfolders, called "curated" and "original respectively
* The script, "PATIENT-1-clean_data.ipynb" is stored in folder "project_calc/python"
* The original data 'Tianlin_GFR_CKD_EPI_31082020.xls' is folder "project_calc/data/original"
* The cleaned data (output) will be  stored in "project_calc/data/curated/" once it is generated from the script.

In case you adopt the same folder setting, you do not have to change anything from the script, just run it!

<br> Alternatively, if you have the input data and this script in the SAME folder, you do not need to specify f1 and f2,because current path=default path. So: just change the two lines:
* x1=CLEAN1(f1+'Tianlin_GFR_CKD_EPI_31082020.xls') to x1=CLEAN1('Tianlin_GFR_CKD_EPI_31082020.xls')
* x1.export_excel(f2+'what-ever-name-you-like.xlsx') to x1.export_excel('what-ever-name-you-like.xlsx')    


In [1]:
import pandas as pd
import numpy as np

from set_path import *

In [2]:
def gfr_female(x): #calculate eGFR-EPI for female
    kappa=0.7
    alpha=-0.329
    return 141*np.power(min(x['S_Kreatinin']/kappa,1),alpha)*np.power(max(x['S_Kreatinin']/kappa,1),-1.209)\
                    *np.power(0.993,x['Age'])*1.018

def gfr_male(x): #calculate eGFR-EPI for male
    kappa=0.9
    alpha=-0.411
    return 141*np.power(min(x['S_Kreatinin']/kappa,1),alpha)*np.power(max(x['S_Kreatinin']/kappa,1),-1.209)\
                    *np.power(0.993,x['Age'])


In [3]:
class CLEAN1:
    def __init__(self,input1):
        self.df1=pd.read_excel(input1,index_col=1)
        # display original no. of rows and columns of the table
        print ('It contains',self.df1.shape[0],'rows and', self.df1.shape[1],'columns')
        # display column names
#         print (self.df1.columns.tolist())
#         print (self.df1.head)
        # display study names in column 'fidStudie'
#         print (self.df1.fidStudie.unique())
#         print (self.df1.gender.unique())
    def remove_nan(self): # remove one patient without eGFR information

        self.df1=self.df1[self.df1.GFR_CKD_EPI.notnull()]
        print ('It contains',self.df1.shape[0],'rows and', self.df1.shape[1],'columns')
        
    def remove_study(self): # remove unwanted studies 
        
        # name of the unwanted studies 
        studies=['FROG_ICU','CIHF','PCHF','MTWA','Haubitz_geblindet_II','Stroke'] 
        self.df1=self.df1[~self.df1.fidStudie.isin(studies)]
        print ('It contains',self.df1.shape[0],'rows and', self.df1.shape[1],'columns')
        
    def remove_s_creatinine_bySam(self):
        
        slist=['1971-Comper_Diabetes-Urin-4625',
                '591-FSGS-Urin-2037',
                '5-FSGS-Urin-130',
                '5-FSGS-Urin-692']
        
        self.df1=self.df1[~self.df1.index.isin(slist)]
        print ('It contains',self.df1.shape[0],'rows and', self.df1.shape[1],'columns')
        
    def remove_dialysis(self,input1): # remove NTCVD patients with dialysis='Y'
        
        df1=pd.read_excel(input1) #input from Guilia
        
        dialysis=df1.loc[df1['Dialysis Y/N']=='Y','SampleID'].tolist() #list ID of patients with dialysis=I
        
        self.df1=self.df1[~self.df1.index.isin(dialysis)] # exclude those in the above list
        
        print ('It contains',self.df1.shape[0],'rows and', self.df1.shape[1],'columns')
    
    def remove_age_below18(self): # remove patient below 18
        
        self.df1=self.df1[self.df1.Age>=18] # select age>=18
        
        print ('It contains',self.df1.shape[0],'rows and', self.df1.shape[1],'columns')
    
    def remove_egfr_above(self,n): # remove patients with eGFR > n
        
        self.df1=self.df1[self.df1.GFR_CKD_EPI<=n] # select eGFR <=n
        
        print ('It contains',self.df1.shape[0],'rows and', self.df1.shape[1],'columns')
        
    def egfr_predictionHBI(self): # correct eGFR of predictions_HBI patients
        
        # mask1 identifies the predictions_HIB patients with extra small s_creatinine
        mask1=(self.df1.fidStudie=='Predictions_HIB')&(self.df1.S_Kreatinin<0.1)
        
        # convert s_creatinine for mask1 patients from mmol/L to mg/dL
        self.df1.loc[mask1,'S_Kreatinin']*=88.42
#         print (self.df1.loc[mask1,['fidStudie','OrigID', 'S_Kreatinin']])

        # calculate egfr-EPI of predictionHBI patients with sexfemale
        mask_f=(self.df1.fidStudie=='Predictions_HIB')&(self.df1.gender=='weiblich')
        self.df1.loc[mask_f,'GFR_CKD_EPI']=self.df1.loc[mask_f].apply(gfr_female,axis=1)
#         print (self.df1.loc[mask_f,['Age','gender', 'S_Kreatinin','GFR_CKD_EPI']])

        # calculate egfr-EPI of predictionHBI with sex male
        mask_m=(self.df1.fidStudie=='Predictions_HIB')&(self.df1.gender=='männlich')
        self.df1.loc[mask_m,'GFR_CKD_EPI']=self.df1.loc[mask_m].apply(gfr_male,axis=1)
#         print (self.df1.loc[mask_m,['Age','gender', 'S_Kreatinin','GFR_CKD_EPI']])
        
    def merge_diseases(self,input2):# merge diseases into groups
        
        # load group_disease from Sam but omit the last row "Gesamtergebnis"
        df2=pd.read_excel(input2,index_col=0).iloc[:-1,:]
#         print (df2.columns.tolist())

        """create a dictionary with six groups according to Sam
                        group name = column name
        ['Exclude','Control', 'Diabetes/Obesity/metabolic syndrome', 
        'Chronic kidney disease', 'CVD/Hypertension', 'Other diseases', 
        'More Information Needed']
        """ 
        disease_dict=df2.to_dict()
#         print (disease_dict)

        # remove two groups we do not need
        del disease_dict['More Information Needed']
        del disease_dict['Control']
        
        """merge diseases into five groups: 
        ['Exclude','Diabetes/Obesity/metabolic syndrome', 
        'Chronic kidney disease', 'CVD/Hypertension', 'Other diseases']
        """ 
        for i in disease_dict:
            self.df1[i]=np.where(self.df1[[k for k,v in disease_dict[i].items() if v==1.0]].any(axis=1),
                                 1,np.nan)
            
        # remove subdiseases columns
        self.df1.drop(list(df2.index),axis=1,inplace=True)
        self.df1.drop('Gesamtergebnis',axis=1,inplace=True)
        
        print ('It contains',self.df1.shape[0],'rows and', self.df1.shape[1],'columns')
        
    def exclude_diseases(self): #exclude patients with 1.0 in "exclude" columns
        
        self.df1=self.df1[self.df1['Exclude'].isnull()]
        self.df1.drop('Exclude',axis=1,inplace=True) # delete the "exclude" columns
        
        print ('It contains',self.df1.shape[0],'rows and', self.df1.shape[1],'columns')
    def update_Calcification(self,input1):
        print ('Before updating Calcification:')
        print ('There are',self.df1['Diabetes/Obesity/metabolic syndrome'].count(), 'non-null rows of diabetes')
        print ('There are',self.df1['CVD/Hypertension'].count(), 'non-null rows of CVD')
        print ('There are',self.df1['Chronic kidney disease'].count(), 'non-null rows of CKD')
        
    
        # read Calcification data from Sam, notice column 'NTCVD-ID' must be converted to str for merge to work
        df2=pd.read_excel(input1,dtype={'DNAtx Patient Number':str})
    
                                                              
        df2.rename({'Diabetes_no0_yes1':'Diabetes/Obesity/metabolic syndrome',
                   'CVD_No0_Yes1':'CVD/Hypertension'},axis=1,inplace=True)
        
    
        # create a temporary df3 that:
        # its index is 'SampleID'
        # it merges GB file with original data based on 'NTCVD-ID'
        df3=pd.DataFrame(self.df1.iloc[:,0]).reset_index()\
                        .merge(df2,left_on='OrigID',right_on='DNAtx Patient Number')\
                        .set_index('SampleID')

        # update original data based on temporary df3
        self.df1.update(df3)
        
        # set all as having CVD
        self.df1.loc[self.df1['fidStudie']=='Calcification','Chronic kidney disease']=1
        print ('After updating Calcification:')
        print ('There are',self.df1['Diabetes/Obesity/metabolic syndrome'].count(), 'non-null rows of diabetes')
        print ('There are',self.df1['CVD/Hypertension'].count(), 'non-null rows of CVD')
        print ('There are',self.df1['Chronic kidney disease'].count(), 'non-null rows of CKD')

        
    def insert_col_from_sql(self,input1,colname): # insert colname from sql-generated xls by merging index
        
        uc=pd.read_excel(input1,index_col=0)
#         print ('shape of input1',uc.shape)
#         print (uc.columns.tolist())
        
        # keep the UNIQUE index with earliest date as possible
        uc.sort_values(['Datum','Einheit'],inplace=True)
        uc = uc[~uc.index.duplicated(keep='first')]
        
        uc.drop(['Parameter', 'Datum'],inplace=True,axis=1)
        
        uc.rename({'Wert':colname,
                      'Einheit':'unit_'+colname},axis=1, inplace=True)
        
        # merge input1 with original data based on index
        self.df1=self.df1.join(uc,how='left')
        
        print ('It contains',self.df1.shape[0],'rows and', self.df1.shape[1],'columns')
        
        print ('Summary:')
        print ('Type of units',self.df1['unit_'+colname].unique().tolist())
        print ('There are',self.df1[colname].count(), 'non-null rows of',colname)
        self.df1.drop('unit_'+colname,axis=1,inplace=True)

        
    def update_NTCVD(self,input1): #update NTCVD patients act. data provided by GB
        
        print ('Before updating NTCVD:')
        print ('There are',self.df1['U_Albumin'].count(), 'non-null rows of U_Albumin')
        print ('There are',self.df1['acr'].count(), 'non-null rows of acr')
    
        # read NTCVD data from GB, notice column 'NTCVD-ID' must be converted to str for merge to work
        df2=pd.read_excel(input1,sheet_name=1,dtype={'NTCVD-ID':str})

        # for the columns to be updated, rename them in GB file so that they are the same as original data
        df2.rename({'UrinAlbumin (UK) mg/dl':'U_Albumin',
                   'UrinAlbumin mg/gKrea (UK)':'acr'},axis=1, inplace=True)
        
#         print (df2.shape)
#         print (df2.columns.tolist())
#         print (df2.head())

        # create a temporary df3 that:
        # its index is 'SampleID'
        # it merges GB file with original data based on 'NTCVD-ID'
        df3=pd.DataFrame(self.df1.iloc[:,0]).reset_index()\
                        .merge(df2,left_on='OrigID',right_on='NTCVD-ID')\
                        .set_index('SampleID')

        # update original data based on temporary df3
        self.df1.update(df3)
        
        print ('After updating NTCVD:')
        print ('There are',self.df1['U_Albumin'].count(), 'non-null rows of U_Albumin')
        print ('There are',self.df1['acr'].count(), 'non-null rows of acr')

        # uncomment to print the updated columns 
    #     print (df1.loc[df1.fidStudie=='NTCVD',['U_Albumin','acr']])
    
    def replace_0_by_nan(self,colname): # replace 0 by nan in any designated column
        
        print ('number of nan before replacement:',self.df1[colname].isnull().sum())
        print ('number of 0 before replacement:',(self.df1[colname]==0).sum())
        
        self.df1[colname]=self.df1[colname].replace(0,np.nan)
        
        print ('number of nan after replacement:',self.df1[colname].isnull().sum())
        print ('number of 0 after replacement:',(self.df1[colname]==0).sum())
        
    def replace_0_by_min(self,colname): # replace 0 by smallest non-zero value in a groupwise manner
        
        print ('number of nan before replacement:',self.df1[colname].isnull().sum())
        print ('number of 0 before replacement:',(self.df1[colname]==0).sum())
        
        # covert nan to inf, then 0 to nan
        self.df1[colname] = self.df1[colname].replace(np.nan,np.inf).replace(0,np.nan)
        
        # fill nan with smallest value in group 'fidStudie'
        self.df1[colname]=self.df1[colname].fillna(self.df1.groupby('fidStudie')[colname].transform('min'))
        
        # convert inf bach to nan, so that we preserve all inf records
        self.df1[colname]=self.df1[colname].replace(np.inf,np.nan)
        
        print ('number of nan after replacement:',self.df1[colname].isnull().sum())
        print ('number of 0 after replacement:',(self.df1[colname]==0).sum())
        
    def assign_sex(self): #assign boolean to sex, m=0, f=1
        
        self.df1['gender']=np.where(self.df1['gender']=='männlich',1,0)
        
    def albuminuria_group(self):
        self.df1['normalalbuminuria']=np.where(self.df1['acr']<30,1,0)-self.df1['acr']+self.df1['acr']
        self.df1['microalbuminuria']=np.where(((self.df1['acr']<300)&(self.df1['acr']>=30)),1,0)-self.df1['acr']+self.df1['acr']
        self.df1['macroalbuminuria']=np.where(self.df1['acr']>=300,1,0)-self.df1['acr']+self.df1['acr']
        
        print ('number of 1 in normalalbuminuria:',(self.df1['normalalbuminuria']==1).sum())
        print ('number of 0 in normalalbuminuria:',(self.df1['normalalbuminuria']==0).sum())
        print ('number of 1 in microalalbuminuria:',(self.df1['microalbuminuria']==1).sum())
        print ('number of 0 in microalalbuminuria:',(self.df1['microalbuminuria']==0).sum())
        print ('number of 1 in macroalalbuminuria:',(self.df1['macroalbuminuria']==1).sum())
        print ('number of 0 in macroalalbuminuria:',(self.df1['macroalbuminuria']==0).sum())
        
    def egfr_group(self,thres,labels):
        
        thres=[0]+thres+[np.inf]
        
        self.df1['egfr_group']=self.df1[['GFR_CKD_EPI']].apply(lambda x : pd.cut(x,thres,labels=labels))
        print (self.df1.groupby('egfr_group')['GFR_CKD_EPI'].describe())
                                                        
    
    def export_excel(self,output1):
        print ('The cleaned table can be found at',output1)
        self.df1.to_excel(output1)
    def export_csv(self,output1):
        print ('The cleaned table can be found at',output1)
        self.df1.to_csv(output1)
    def export_csv_train_test(self,output1,output2):
        maskc=self.df1['fidStudie']=='Calcification'
        
        # training dataset do not have "Calcification"
        train=self.df1[~maskc]
        train.to_csv(output1)
        print ('The cleaned train data can be found at',output1)
        print ('number of rows in train:',len(train))
        
        # test dataset contains only "Calcification"
        test=self.df1[maskc]
        test.to_csv(output2)
        print ('The cleaned test data can be found at',output2)
        print ('number of rows in test:',len(test))
        

In [4]:
x1=CLEAN1(f1+'Tianlin_GFR_CKD_EPI_31082020.xls')
x1.remove_nan()
x1.remove_s_creatinine_bySam()
x1.remove_study()
x1.remove_dialysis(f1+'checkedByGB_Tianlin_ntcvd.xlsx')
x1.remove_age_below18()
x1.remove_egfr_above(150)
x1.egfr_predictionHBI()
x1.merge_diseases(f1+'disease_group_SamHobson_PS_SH.xlsx')
x1.exclude_diseases()
x1.update_Calcification(f1+'Karltx_Matching_UrineSamples_Hannover_v4 (2).xlsx') # update diseases in study "Calcification"
x1.insert_col_from_sql(f1+'acr_unit_date.xls','acr')
x1.update_NTCVD(f1+'Tianlin_ntcvd_all_checkbyGB.xlsx')
x1.replace_0_by_min('U_Albumin')
x1.assign_sex()
x1.albuminuria_group() #assign normal/micro/macro albuminuria
x1.egfr_group(thres=[30,60,90],labels=[3,2,1,0]) #label 4 eGFR groups, larger the eGFR, smaller the label
# to export all patients
x1.export_csv(f2+'Tianlin_GFR_CKD_EPI_cleaned_20201005.csv')
# to export train and test separately
#x1.export_csv_train_test(f2+'train_20201005.csv',f2+'test_20201005.csv')

It contains 14345 rows and 84 columns
It contains 14344 rows and 84 columns
It contains 14340 rows and 84 columns
It contains 10646 rows and 84 columns
It contains 10636 rows and 84 columns
It contains 8387 rows and 84 columns
It contains 8114 rows and 84 columns
It contains 8114 rows and 17 columns
It contains 7939 rows and 16 columns
Before updating Calcification:
There are 3305 non-null rows of diabetes
There are 1697 non-null rows of CVD
There are 1373 non-null rows of CKD
After updating Calcification:
There are 3372 non-null rows of diabetes
There are 1758 non-null rows of CVD
There are 1413 non-null rows of CKD
WARNING *** file size (15736254) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
It contains 7939 rows and 18 columns
Summary:
Type of units [nan, 'mg/g']
There are 1545 non-null rows of acr
Before updating NTCVD:
There are 3943 non-null rows of U_Albumin
There are 1545 non-null rows of acr
After updating NTC